In [10]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [11]:
from TexSoup import TexSoup

def section_split(tex):
    """returns: List of tuples (section_name, section_text)"""
    try:
        soup = TexSoup(tex)

        doc_text = tex

        sections = []

        sections.append(("title", soup.title.contents[0])) if soup.title else None
        sections.append(("abstract", soup.abstract.contents[0])) if soup.abstract else None
        sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None


        # extract latex sections and corresponding text
        prev_section = "pre"

        for node in soup.find_all("section"):
            section_text, doc_text = doc_text.split(str(node), 1)
            sections.append((prev_section, section_text))
            prev_section = node.contents[0]
        sections.append((prev_section, doc_text))
        return sections
    except:
        return {"full": tex}
    


In [12]:
import ollama
def pass_to_ollama(prompt, model):
    try:
        res = ollama.generate(model=model, prompt=prompt, options={"temperature": 0})
        return res["response"]
    except Exception as ex:
        print(ex)
        return f"ollama error: {ex}"

In [13]:
import json
import tqdm
from tqdm import tqdm
from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
from datetime import datetime


def extract_tdms(tex):
    return f'If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.\
     \n \
    Text: \n {tex} \n \
    Return the tasks, datasets, metrics and scores as reported in the text in a JSON array, \
    for example [{{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, {{"Task": "example Task 1","Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}] and provide the JSON Array only. \n \
    Do not include precision information in the reported score. \
    Entries: '

def convert_tdms_to_tuple(model_output_parsed):
    tuples = []
    for item in l:
        try:
            t = ((item["Task"], item["Dataset"],item["Metric"],item["Score"]))
            tuples.append(t)
        except:
            # parse error, ignore instance
            pass

def format_tdms(tuples):
    """make unique, format as string"""
    unique = set(tuples)
    dicts = [{"LEADERBOARD": {
        "Task": t,
        "Dataset":d,
        "Metric":m,
        "Score":s
    }} for t,d,m,s in unique]
    return str(dicts)



# Run on validation data

train_dataset = TDMSDataset(PATH.VAL)
model = "llama3:70b"

run_id = f"baseline-tdms-valid_{model.replace(':', '_')}-{datetime.now().strftime('%m%d%Y-%H%M%S')}"
llama3_fn = lambda prompt: pass_to_ollama(prompt, model)


logger = LogResult(run_id)
indexes = 1

for i in tqdm(range(indexes)):
    f, tex, ground_truth = train_dataset.__getitem__(100)
    found_tdms = []
    sections = section_split(tex)
    for section_name, section_text in sections:
        response = llama3_fn(extract_tdms(section_text))

        try:
            response = json.loads(response)
            parsed = convert_tdms_to_tuple(response)
            found_tdms= [*found_tdms, *parsed]
        except:
            pass # no tuples found in section

    # flatten across sections
    results = [section_tdms for section in found_tdms for section_tdms in section]
    if not results:
        annotation = UNANSWERABLE # results are empty -> unanswerable
    else:
        # dedupe and format
        results = format_tdms(results, do_write=False)
    # log
    logger.log(f, results)

df = logger.save()


df

ImportError: cannot import name 'LogResult' from 'src.dataset' (/home/user/code/nlp2024_ClefTask4SOTA/src/dataset.py)

In [ ]:
results



: 

: 

: 

In [ ]:
set([(r["Task"], r["Dataset"],r["Metric"],r["Score"]) for r in result])

: 

: 

In [ ]:
ground_truth

: 

: 

# Evaluation

In [ ]:
# pct of success

df["success"] =df["ground_truth"]==df["pred"]
len(df[df["success"]]) / len(df)

# .76

: 

: 

In [ ]:
# confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(df["ground_truth"], df["pred"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

: 

: 

In [ ]:
# random trial
import random

num_true = len(df[df["ground_truth"]])
num_tot = len(df)
pct_true = num_true / num_tot


df["random"] = random.uniform(0, 1)
df["random_success"] = df["ground_truth"] == (df["random"] <= pct_true)
len(df[df["random_success"]]) / len(df)

# duh..
# .634 (random with distribution), .758 (baseline llama)
# codalab: .53 (llama 2 baseline), .83 (top score)

: 

: 

# Run on Test

In [ ]:
# run on train data to evaluate method
from tqdm import tqdm
import pandas as pd
from src.dataset import BinaryTDMSDataset, PATH, write_annotation_file, UNANSWERABLE
from datetime import datetime


test_dataset = BinaryTDMSDataset(PATH.TEST)
model = "llama3:70b"

run_id = f"baseline-test_{model.replace(':', '_')}-{datetime.now().strftime('%m%d%Y-%H%M%S')}"
llama3_fn = lambda prompt: pass_to_ollama(prompt, model)

results = []
for i in tqdm(range(len(test_dataset))):
# for i in tqdm(range(1)):
    f, tex, _ = test_dataset.__getitem__(i)
    pred = section_wise_detection(tex, llama3_fn)
    write_annotation_file(run_id, f, UNANSWERABLE if not pred else "Something was found")
    # print(f"{f}: {pred} ({ground_truth})")
    results.append((f, pred))


df = pd.DataFrame(results)
df.columns = ["file", "pred"]
df.to_feather(f"results/{run_id}/df.feather")

: 

: 

In [ ]:
# # TODO: Test set evalutation

# from src.dataset import BinaryTDMSDataset, PATH

# test_dataset = BinaryTDMSDataset(PATH.TEST)

# def get_index(folder):
#     return [i for i, t, j in test_dataset.all_paths].index(folder)

# i, tex, jsn = test_dataset.__getitem__(get_index("0706.0014"))


: 

: 

In [ ]:
res_path = "first_baseline_70b.feather"

import pandas as pd

df = pd.read_feather(res_path)



: 

: 

In [ ]:
len(df)

: 

: 

In [ ]:
len(df[df["pred"]])

: 

: 